In [7]:
import os 
import heapq
class BinaryTreeNode:
    def __init__(self,value,freq):
        self.value = value
        self.freq = freq
        self.right = None
        self.left = None
        
    def __lt__(self,other):
        return self.freq < other.freq
    
    def __equal__(self,other):
        return self.freq == other.freq 

class HuffmanCoding:
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__code = {}
        self.__reverseCodes ={}
    
    def __make_frequency(self,text):
        freq_dict = {}
        
        for char in text:
            if char not in freq_dict:
                freq_dict[char] = 0
            freq_dict[char] += 1
            
        return freq_dict
        
    def __buildHeap(self,freq_dict):
        for key in freq_dict:
            frequency = freq_dict[key]
            binary_treeNode = BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap,binary_treeNode)
            
    def __buildTree(self):
        while(len(self.__heap) > 1):
            bin1 = heapq.heappop(self.__heap)
            bin2 = heapq.heappop(self.__heap)
            sum_freq = bin1.freq + bin2.freq
            newNode = BinaryTreeNode(None , sum_freq)
            newNode.left = bin1
            newNode.right = bin2
            heapq.heappush(self.__heap,newNode)
        return 
    def buildCodeHelper(self,root,currBits):
        if root is None:
            return
        
        if root.value is not None:
            self.__code[root.value] = currBits
            self.__reverseCodes[currBits] = root.value
            return 
        
        self.buildCodeHelper(root.left,currBits+"0")
        self.buildCodeHelper(root.right,currBits+"1")
        
    
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.buildCodeHelper(root,"")
    
    def __getEncoded(self,text):
        encoded_text = ""
        for char in text:
            encoded_text += self.__code[char]
        return encoded_text
    
    def __getPadded(self,encoded_text):
        padded_amount = 8 - (len(encoded_text)%8)
        
        for i in range(padded_amount):
            encoded_text +="0"
        padded_info = "{0:08b}".format(padded_amount)
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text
    
    def __getBytesArray(self,padded_encoded_text):
        
        array =[]
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i:i+8]
            array.append(int(byte,2))
        return array
        
        
    
    def compress(self):
        #get file from path
        file_name,file_extension = os.path.splitext(self.path)
        output_path = file_name + '.bin'
        
        with open(self.path , 'r+') as file, open(output_path,'wb') as output:
            
            text = file.read()
            text = text.rstrip()
            freq_dict = self.__make_frequency(text)
                      # Construct heap from freq Dict
            self.__buildHeap(freq_dict)
        
        #construct binary tree from heap
            self.__buildTree()
        
        #Build Codes from BinaryTree
            self.__buildCodes()
        
        #creating encoded Text using binary code
            encoded_text = self.__getEncoded(text)
        
        #pad this encoded text
            padded_encoded_text = self.__getPadded(encoded_text)
        
            bytes_array = self.__getBytesArray(padded_encoded_text)
        
        #return this binary file as output
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)
        print('Compressed')
        return output_path
    
    def __removePadding(self,text):
        padded_info = text[:8]
        extra_padding = int(padded_info,2)
        
        text = text[8:]
        text_after_padding_removed = text[:1*extra_padding]
        return text_after_padding_removed
    def __decodeText(self,text):
        decode_text = ""
        current_bits = ""
        
        
        for bit in text:
            current_bits+=bit
            if current_bits in self.__reverseCodes:
                character =self.__reverseCodes[current_bits]
                decode_text +=character
                current_bits = ""
        return decode_text
    def decompress(self,input_path):
        filename,fileextension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + ".txt"
        with open(input_path, 'rb') as file, open(output_path,'w') as output:
            bit_string = ""
            byte = file.read(1)
            
            while byte:
                byte = ord(byte)
                bits  = bin(byte)[2:].rjust(8,'0')
                bit_string += bits
                byte = file.read(1)
                
            actual_text = self.__removePadding(bit_string)
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text)
        return output_path
             
            

        
        
path = '/Users/Shubham/Downloads/sample.txt'

file=HuffmanCoding(path)
print('Compressing')
compressed_path=file.compress()
print('Location of compressed file : ',compressed_path)
print()
print('Decompressing')
decompressed_path=file.decompress(compressed_file_path)
print('Location of decompressed file : ',decompressed_path)

Compressing
Compressed
Location of compressed file :  /Users/Shubham/Downloads/sample.bin

Decompressing
Location of decompressed file :  /Users/Shubham/Downloads/sample_decompressed.txt
